# fit, predict and submit (xgb+random forest)

In this notebook I use the Extreme Gradient Boosting to make my predictions. This notebook also includes the tuning the xgb classifier.

In [2]:
from __future__ import division
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import matplotlib 
import numpy as np
# Enable inline plotting
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD

from sklearn.manifold import TSNE

from sklearn import preprocessing

from urllib import urlopen
from bs4 import BeautifulSoup
import nltk 

from dateutil import parser

import pyprind

from difflib import SequenceMatcher

import seaborn as sns
sns.set(style="darkgrid", palette="muted")
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('display.notebook_repr_html', True)
plt.rcParams['figure.figsize'] = 10, 6

import warnings
warnings.filterwarnings('ignore')


from sklearn.cross_validation import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score,log_loss
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #513
  (fname, cnt))


In [3]:
from sklearn import preprocessing
def normalize(df, features):
    for k in features:
        df[k] = preprocessing.MinMaxScaler().fit_transform(df[k])
    return df

In [4]:
df_train = pd.read_csv('preprocessing_train_3.csv')
df_test = pd.read_csv('preprocessing_test_3.csv')
features = list(pd.read_csv('features.csv')['0'])
#features = list(set(df_train.columns.values).intersection(df_test.columns.values))
print str(len(features)) + ' features'
print features

destinies = list(set(df_train['OutcomeType']))
labelEncoderDestinies = preprocessing.LabelEncoder()
labelEncoderDestinies.fit(destinies)

for d in destinies:
    df_train[d] = df_train['OutcomeType'].map(lambda x: int(x==d) )
df_train[features][:3]

30 features
['Minute', 'Hour', 'isNotIntact', 'Day', 'Age', 'Month', 'AgeInYears', 'HasName', 'AgeInMonths', 'BreedScore', 'isFemale', 'AgeInWeeks', 'isCat', 'Color=Black', 'Color=White', 'ColorCount', 'Breed=Domestic Shorthair', 'Color=Brown', 'isMix', 'Color=Tan', 'Breed=Pit Bull', 'Color=Blue', 'Breed=Chihuahua Shorthair', 'BreedCount', 'Color=Red', 'Breed=Labrador Retriever', 'Color=Brown Tabby', 'Color=Brown Brindle', 'Color=Tricolor', 'Breed=German Shepherd']


,Minute,Hour,isNotIntact,Day,Age,Month,AgeInYears,HasName,AgeInMonths,BreedScore,...,Breed=Pit Bull,Color=Blue,Breed=Chihuahua Shorthair,BreedCount,Color=Red,Breed=Labrador Retriever,Color=Brown Tabby,Color=Brown Brindle,Color=Tricolor,Breed=German Shepherd
0,22,18,1,12,365,2,1,1,0,0.857143,...,0,0,0,1,0,0,0,0,0,0
1,44,12,1,13,365,10,1,1,0,0.823529,...,0,0,0,1,0,0,0,0,0,0
2,28,12,1,31,730,1,2,1,0,0.000000,...,1,1,0,1,0,0,0,0,0,0


In [5]:
print[f for f in features if 'Color=' not in f and 'Breed=' not in f]

['Minute', 'Hour', 'isNotIntact', 'Day', 'Age', 'Month', 'AgeInYears', 'HasName', 'AgeInMonths', 'BreedScore', 'isFemale', 'AgeInWeeks', 'isCat', 'ColorCount', 'isMix', 'BreedCount']


In [6]:
# prepare the data for XGB
# normalize
df_train = normalize(df_train, features)
df_test = normalize(df_test, features)
target = destinies
predicators = features

df_train['type'] = df_train['OutcomeType'].map( labelEncoderDestinies.transform )
y = df_train['type']
X = df_train[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print str(X_train.shape[1]) + ' features'
print str(X_train.shape[0]) + ' rows\n'

30 features
21383 rows



In [7]:
# quick evaluation of the model
train_X = X_train.values
train_Y = y_train.values

test_X = X_test.values
test_Y = y_test.values

xg_train = xgb.DMatrix( train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

# setup parameters for xgboost
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.12
param['max_depth'] = 8
param['min_child_weight'] = 3
param['silent'] = 1
param['gamma'] = 0.5
param['seed'] = 44
param['alpha'] = 0.001
param['lambda'] = 5
param['num_class'] = len(destinies)

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 100
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round )#, watchlist );
yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], len(destinies) )
ylabelpred = np.argmax(yprob, axis=1)

In [8]:
print len(ylabelpred)
print ('predicting, classification precision=%f' % (1 - sum( int(ylabelpred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))

5346
predicting, classification precision=0.689862


# tuning
## Tune max_depth and min_child_weight

In [134]:
max_depths = range(3,10,2)
min_child_weights = range(1,6,2)
results = []
for depth in max_depths:
    for child_weight in min_child_weights:
        print "max_depth: " + str(depth),
        print "min_child_weight: " + str(child_weight),
        print "seed: ",
        scores = []
        for i in range(3):
            print i,
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = i)
            train_X = X_train.values
            train_Y = y_train.values
            test_X = X_test.values
            test_Y = y_test.values
            xg_train = xgb.DMatrix( train_X, label=train_Y)
            xg_test = xgb.DMatrix(test_X, label=test_Y)
            # setup parameters for xgboost
            param = {}
            # use softprob multi-class classification
            param['objective'] = 'multi:softprob'
            # scale weight of positive examples
            param['eta'] = 0.2
            param['max_depth'] = depth
            param['min_child_weight'] = child_weight
            param['silent'] = 1
            param['gamma'] = 0
            param['seed'] = 44
            param['num_class'] = len(destinies)
            watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
            num_round = 100
            # do the same thing again, but output probabilities
            param['objective'] = 'multi:softprob'
            bst = xgb.train(param, xg_train, num_round) #, watchlist );
            yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], len(destinies) )
            ylabelpred = np.argmax(yprob, axis=1)
            score = ((1 - sum( int(ylabelpred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))
            scores.append(score)
        print "score:",
        print np.mean(scores)
        results.append( {'score' : np.mean(scores) , 'scores' : scores, 'max_depth' : depth, 'min_child_weight' :  child_weight } )

max_depth: 3 min_child_weight: 1 seed:  0 1 2 score: 0.680259383963
max_depth: 3 min_child_weight: 3 seed:  0 1 2 score: 0.679760568649
max_depth: 3 min_child_weight: 5 seed:  0 1 2 score: 0.68050879162
max_depth: 5 min_child_weight: 1 seed:  0 1 2 score: 0.683189923931
max_depth: 5 min_child_weight: 3 seed:  0 1 2 score: 0.684436962215
max_depth: 5 min_child_weight: 5 seed:  0 1 2 score: 0.681755829904
max_depth: 7 min_child_weight: 1 seed:  0 1 2 score: 0.683938146901
max_depth: 7 min_child_weight: 3 seed:  0 1 2 score: 0.683751091158
max_depth: 7 min_child_weight: 5 seed:  0 1 2 score: 0.68555929667
max_depth: 9 min_child_weight: 1 seed:  0 1 2 score: 0.685871056241
max_depth: 9 min_child_weight: 3 seed:  0 1 2 score: 0.686681631126
max_depth: 9 min_child_weight: 5 seed:  0 1 2 score: 0.685309889014


In [138]:
for r in sorted( results , key = lambda x : np.mean(x['score']) , reverse = True)[:3]:
    #r ['score'] = np.mean(r['score'])
    print r

{'score': 0.68668163112607561, 'max_depth': 9, 'min_child_weight': 3, 'scores': [0.6835016835016835, 0.69248035914702588, 0.68406285072951745]}
{'score': 0.68587105624142664, 'max_depth': 9, 'min_child_weight': 1, 'scores': [0.68163112607557053, 0.69117096894874674, 0.68481107369996264]}
{'score': 0.68555929667040783, 'max_depth': 7, 'min_child_weight': 5, 'scores': [0.68499812944257388, 0.69322858211747107, 0.67845117845117842]}


best: 8,5

## Tune gamma 

In [41]:
gammas = [i/100.0 for i in range(0,5)]
results = []
for gamma in gammas:
    print "gamma: " + str(gamma),
    print "seed: ",
    scores = []
    for i in range(3):
        print i,
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = i)
        train_X = X_train.values
        train_Y = y_train.values
        test_X = X_test.values
        test_Y = y_test.values
        xg_train = xgb.DMatrix( train_X, label=train_Y)
        xg_test = xgb.DMatrix(test_X, label=test_Y)
        # setup parameters for xgboost
        param = {}
        # use softprob multi-class classification
        param['objective'] = 'multi:softprob'
        # scale weight of positive examples
        param['eta'] = 0.2
        param['max_depth'] = 8
        param['min_child_weight'] = 5
        param['silent'] = 1
        param['gamma'] = gamma
        param['seed'] = 44
        param['num_class'] = len(destinies)

        watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
        num_round = 100
        # do the same thing again, but output probabilities
        param['objective'] = 'multi:softprob'
        bst = xgb.train(param, xg_train, num_round) #, watchlist );
        yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], len(destinies) )
        ylabelpred = np.argmax(yprob, axis=1)
        score = ((1 - sum( int(ylabelpred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))
        scores.append(score)
    print "score:",
    print np.mean(scores)
    results.append( {'score' : np.mean(scores) , 'scores' : scores, 'gamma' : gamma} )

gamma: 0.0 seed:  0 1 2 score: 0.686245167727
gamma: 0.01 seed:  0 1 2 score: 0.683439331587
gamma: 0.02 seed:  0 1 2 score: 0.68287816436
gamma: 0.03 seed:  0 1 2 score: 0.684686369872
gamma: 0.04 seed:  0 1 2 score: 0.682316997132


gamma best values: 0

## tune alpha

In [11]:
alphas = [1e-5, 1e-2, 0.1, 1, 100]
results = []
for alpha in alphas:
    print "alpha: " + str(alpha),
    print "seed: ",
    scores = []
    for i in range(3):
        print i,
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = i)
        train_X = X_train.values
        train_Y = y_train.values
        test_X = X_test.values
        test_Y = y_test.values
        xg_train = xgb.DMatrix( train_X, label=train_Y)
        xg_test = xgb.DMatrix(test_X, label=test_Y)
        # setup parameters for xgboost
        param = {}
        # use softprob multi-class classification
        param['objective'] = 'multi:softprob'
        # scale weight of positive examples
        param['eta'] = 0.2
        param['max_depth'] = 8
        param['min_child_weight'] = 5
        param['silent'] = 1
        param['gamma'] = 0
        param['alpha'] = alpha
        param['seed'] = 44
        param['num_class'] = len(destinies)

        watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
        num_round = 100
        # do the same thing again, but output probabilities
        param['objective'] = 'multi:softprob'
        bst = xgb.train(param, xg_train, num_round) #, watchlist );
        yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], len(destinies) )
        ylabelpred = np.argmax(yprob, axis=1)
        score = ((1 - sum( int(ylabelpred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))
        scores.append(score)
    print "score:",
    print np.mean(scores)
    results.append( {'score' : np.mean(scores) , 'scores' : scores, 'alpha' : alpha} )

alpha: 1e-05 seed:  0 1 2 score: 0.686245167727
alpha: 0.01 seed:  0 1 2 score: 0.68287816436
alpha: 0.1 seed:  0 1 2 score: 0.683002868188
alpha: 1 seed:  0 1 2 score: 0.683314627759
alpha: 100 seed:  0 1 2 score: 0.674211248285


In [12]:
for r in sorted( results , key = lambda x : np.mean(x['score']) , reverse = True)[:3]:
    print r

{'alpha': 1e-05, 'score': 0.68624516772664912, 'scores': [0.68593340815563031, 0.69191919191919193, 0.68088290310512534]}
{'alpha': 1, 'score': 0.68331462775907215, 'scores': [0.68911335578002242, 0.68555929667040783, 0.67527123082678631]}
{'alpha': 0.1, 'score': 0.68300286818805345, 'scores': [0.68275346053123831, 0.68761690983913204, 0.67863823419378977]}


In [8]:
alphas = [1e-6, 1e-5, 1e-4,1e-3]
results = []
for alpha in alphas:
    print "alpha: " + str(alpha),
    print "seed: ",
    scores = []
    for i in range(3):
        print i,
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = i)
        train_X = X_train.values
        train_Y = y_train.values
        test_X = X_test.values
        test_Y = y_test.values
        xg_train = xgb.DMatrix( train_X, label=train_Y)
        xg_test = xgb.DMatrix(test_X, label=test_Y)
        # setup parameters for xgboost
        param = {}
        # use softprob multi-class classification
        param['objective'] = 'multi:softprob'
        # scale weight of positive examples
        param['eta'] = 0.2
        param['max_depth'] = 8
        param['min_child_weight'] = 5
        param['silent'] = 1
        param['gamma'] = 0
        param['alpha'] = alpha
        param['seed'] = 44
        param['num_class'] = len(destinies)

        watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
        num_round = 100
        # do the same thing again, but output probabilities
        param['objective'] = 'multi:softprob'
        bst = xgb.train(param, xg_train, num_round) #, watchlist );
        yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], len(destinies) )
        ylabelpred = np.argmax(yprob, axis=1)
        score = ((1 - sum( int(ylabelpred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))
        scores.append(score)
    print "score:",
    print np.mean(scores)
    results.append( {'score' : np.mean(scores) , 'scores' : scores, 'alpha' : alpha} )

alpha: 1e-06 seed:  0 1 2 score: 0.686245167727
alpha: 1e-05 seed:  0 1 2 score: 0.686245167727
alpha: 0.0001 seed:  0 1 2 score: 0.684374610301
alpha: 0.001 seed:  0 1 2 score: 0.68244170096


In [10]:
for r in sorted( results , key = lambda x : np.mean(x['score']) , reverse = True)[:3]:
    print r

{'alpha': 1e-06, 'score': 0.68624516772664912, 'scores': [0.68593340815563031, 0.69191919191919193, 0.68088290310512534]}
{'alpha': 1e-05, 'score': 0.68624516772664912, 'scores': [0.68593340815563031, 0.69191919191919193, 0.68088290310512534]}
{'alpha': 0.0001, 'score': 0.68437461030053626, 'scores': [0.68368873924429474, 0.6909839132061355, 0.67845117845117842]}


I use the same process to find the best values for eta (learning rate, lambda and the number of steps)

# -----------------------------------------------------------------------------------
# train on the whole dataset, predict and ouput
# -----------------------------------------------------------------------------------

## xgb only

In [9]:
train_X = df_train[features].values
train_Y = df_train['OutcomeType'].map( labelEncoderDestinies.transform ).values

print train_Y

test_X = df_test[features].values

xg_train = xgb.DMatrix( train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X)

param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.12
param['max_depth'] = 8
param['min_child_weight'] = 3
param['silent'] = 1
param['gamma'] = 0.5
param['seed'] = 44
param['alpha'] = 0.001
param['lambda'] = 5
param['num_class'] = len(destinies)

watchlist = [ (xg_train,'train')]
num_round = 100
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round )
yprob = bst.predict( xg_test )

[3 2 0 ..., 0 4 4]


In [10]:
output_xgb = pd.DataFrame(yprob, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output_xgb.index += 1
output_xgb.to_csv('submission27-xgboost-tuned+breed-score+feature-selection.csv', index_label='ID')
output_xgb[:10]

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.026166,0.002763,0.018962,0.145059,0.807049
2,0.709248,0.000920,0.008063,0.205729,0.076040
3,0.441760,0.004402,0.012202,0.216957,0.324679
4,0.241882,0.005386,0.027004,0.428157,0.297572
5,0.367302,0.001082,0.012561,0.529992,0.089062
6,0.116060,0.001162,0.014366,0.706597,0.161815
7,0.117947,0.085590,0.579883,0.169116,0.047465
8,0.857592,0.024291,0.009626,0.014682,0.093808
9,0.829638,0.001015,0.002007,0.135161,0.032179
10,0.564258,0.000849,0.030421,0.326636,0.077836


### Using multiple classifiers

##### Averaging

I simply use the average of two classifiers 

In [11]:
model = RandomForestClassifier(n_estimators=300, random_state=12, min_samples_split = 3, min_samples_leaf = 2, n_jobs=4)

In [12]:
y = df_train["OutcomeType"]
X = df_train[features]

model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=4,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [13]:
X_test = df_test[features]
pred = model.predict_proba(X_test)

In [14]:
output_rf = pd.DataFrame(pred, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output_rf.index += 1
output_rf[:10]

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.094098,0.001905,0.041184,0.217397,0.645415
2,0.592110,0.000851,0.021890,0.286058,0.099091
3,0.413942,0.000277,0.022840,0.184525,0.378416
4,0.215565,0.003333,0.086486,0.314288,0.380328
5,0.415659,0.000000,0.021419,0.412464,0.150458
6,0.212099,0.000000,0.023368,0.626274,0.138259
7,0.281319,0.007690,0.289633,0.235879,0.185478
8,0.684767,0.006091,0.014948,0.034653,0.259541
9,0.806109,0.000000,0.000000,0.086553,0.107339
10,0.501351,0.000000,0.077330,0.338271,0.083048


In [15]:
voting = 0.5 * (pred + yprob)
output_rf = pd.DataFrame(voting, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output_rf.index += 1
output_rf[:10]

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.060132,0.002334,0.030073,0.181228,0.726232
2,0.650679,0.000885,0.014976,0.245894,0.087566
3,0.427851,0.002340,0.017521,0.200741,0.351547
4,0.228723,0.004360,0.056745,0.371222,0.338950
5,0.391481,0.000541,0.016990,0.471228,0.119760
6,0.164079,0.000581,0.018867,0.666436,0.150037
7,0.199633,0.046640,0.434758,0.202498,0.116472
8,0.771180,0.015191,0.012287,0.024667,0.176675
9,0.817873,0.000507,0.001004,0.110857,0.069759
10,0.532805,0.000424,0.053876,0.332453,0.080442


In [16]:
output_rf.to_csv('submission28-xgboost-tuned+tuned-random_forest+feature-selec.csv', index_label='ID')

#### Ensemble Voting

In [198]:
xgb_clf1 = xgb.XGBClassifier(objective ='multi:softprob', learning_rate = 0.1, max_depth=8,\
                           min_child_weight=3,gamma=0.5,seed=44,reg_alpha=0.001,reg_lambda=5, n_estimators=100)
rdf_clf1 = RandomForestClassifier(n_estimators=300, random_state=12, min_samples_split = 3, min_samples_leaf = 2, n_jobs=4)
rdf_clf2 = RandomForestClassifier(criterion = 'entropy', n_estimators=300, random_state=65, min_samples_split = 3, min_samples_leaf = 2, n_jobs=4)
model = VotingClassifier(estimators=[('xgb1', xgb_clf1) ,('rdf1', rdf_clf1),('rdf2', rdf_clf2) ], voting='soft')
#model = rdf_clf2

##### Quick Cross Validation

In [199]:
df_train = normalize(df_train, features)
df_test = normalize(df_test, features)
y = df_train["OutcomeType"]
features_selected = features[:]
X = df_train[features_selected]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#model =  RandomForestClassifier(n_estimators=500, random_state=12)
print str(X_train.shape[1]) + ' features'
print str(X_train.shape[0]) + ' rows\n'
print 'Fitting...\n'
model.fit(X_train, y_train)
print 'Predicting...\n'
y_pred = model.predict(X_test)
print accuracy_score(y_test, y_pred), ' accuracy.\n'

30 features
21383 rows

Fitting...

Predicting...

0.686494575383  accuracy.



##### Train, Fit, submit

In [200]:
y = df_train["OutcomeType"]
X = df_train[features]

model.fit(X, y)

X_test = df_test[features]
pred = model.predict_proba(X_test)

In [201]:
output_rf = pd.DataFrame(pred, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output_rf.index += 1
output_rf[:10]

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.072526,0.003173,0.039508,0.191886,0.692908
2,0.641971,0.000922,0.019447,0.258328,0.079332
3,0.424993,0.002265,0.015572,0.199556,0.357613
4,0.176696,0.005481,0.070743,0.339483,0.407598
5,0.400142,0.001993,0.022881,0.447320,0.127664
6,0.183452,0.000578,0.020927,0.637496,0.157547
7,0.245004,0.038117,0.396039,0.186210,0.134631
8,0.776743,0.007648,0.015528,0.026041,0.174039
9,0.828567,0.000653,0.000934,0.076845,0.093001
10,0.515204,0.000480,0.055616,0.359472,0.069229


In [202]:
output_rf.to_csv('submission32-xgb+rd-ensemble-voting.csv', index_label='ID')